In [1]:
!git clone https://github.com/linyang-zhh/FQ-ViT.git
%cd FQ-ViT
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ozBb5ynqQfchkNWccbO1qvGD-NRNK9SH' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ozBb5ynqQfchkNWccbO1qvGD-NRNK9SH" -O converted_cifar10-100_500_checkpoint.bin && rm -rf /tmp/cookies.txt

Cloning into 'FQ-ViT'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.
/content/FQ-ViT
--2022-03-15 05:49:23--  https://docs.google.com/uc?export=download&confirm=t&id=1ozBb5ynqQfchkNWccbO1qvGD-NRNK9SH
Resolving docs.google.com (docs.google.com)... 108.177.127.113, 108.177.127.138, 108.177.127.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.127.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kik4l3kcqvaalj7lonbbp2t9sunrp455/1647323325000/17691537378993098219/*/1ozBb5ynqQfchkNWccbO1qvGD-NRNK9SH?e=download [following]
--2022-03-15 05:49:24--  https://doc-0o-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kik4l3kcqvaalj7lonbbp2t9sunrp455/1647323325000/17691537378993098219/*/1ozBb5ynqQfchkNWccbO1qvGD-NRNK9SH?e=download
Res

In [2]:
#if running on a local divice, comment these lines
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
###
### Before running this code cell:
### Add parameter num_classes=10 to the line 552 of models/vit_quant.py file
###

import argparse
import os
import time
import math

from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, SequentialSampler

from models import *
from config import Config


parser = argparse.ArgumentParser(description="FQ-ViT")

# parser.add_argument("model",
#                     choices=['deit_tiny', 'deit_small', 'deit_base', 'vit_base',
#                              'vit_large', 'swin_tiny', 'swin_small', 'swin_base'],
#                     help="model")
# parser.add_argument('data', metavar='DIR',
#                     help='path to dataset')
parser.add_argument("--quant", default=False, action="store_true")
parser.add_argument("--pts", default=False, action="store_true")
parser.add_argument("--lis", default=False, action="store_true")
parser.add_argument("--quant-method", default="minmax",
                    choices=["minmax", "ema", "omse", "percentile"])
parser.add_argument("--calib-batchsize", default=100,
                    type=int, help="batchsize of calibration set")
parser.add_argument("--calib-iter", default=10, type=int)
parser.add_argument("--val-batchsize", default=100,
                    type=int, help="batchsize of validation set")
parser.add_argument("--num-workers", default=16, type=int,
                    help="number of data loading workers (default: 16)")
parser.add_argument("--device", default="cuda", type=str, help="device")
parser.add_argument("--print-freq", default=100,
                    type=int, help="print frequency")
parser.add_argument("--seed", default=0, type=int, help="seed")


def str2model(name):
    d = {'deit_tiny': deit_tiny_patch16_224,
         'deit_small': deit_small_patch16_224,
         'deit_base': deit_base_patch16_224,
         'vit_base': vit_base_patch16_224,
         'vit_large': vit_large_patch16_224,
         'swin_tiny': swin_tiny_patch4_window7_224,
         'swin_small': swin_small_patch4_window7_224,
         'swin_base': swin_base_patch4_window7_224,
         }
    print('Model: %s' % d[name].__name__)
    return d[name]


def seed(seed=0):
    import os
    import sys
    import torch
    import numpy as np
    import random
    sys.setrecursionlimit(100000)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


def main():
    args = parser.parse_args('')
    args.model = 'vit_base'
    args.pretrained_model = 'converted_cifar10-100_500_checkpoint.bin'
    seed(args.seed)
    args.quant = True
    args.pts = True
    args.lis = True

    device = torch.device(args.device)
    cfg = Config(args.pts, args.lis, args.quant_method)
    model = str2model(args.model)(pretrained=False, cfg=cfg)
    checkpoint = torch.load(args.pretrained_model, map_location=args.device)
    model.load_state_dict(checkpoint)
    model = model.to(device)

    # Note: Different models have different strategies of data preprocessing.
    # model_type = args.model.split("_")[0]
    # if model_type == "deit":
    #     mean = (0.485, 0.456, 0.406)
    #     std = (0.229, 0.224, 0.225)
    #     crop_pct = 0.875
    # elif model_type == 'vit':
    #     mean = (0.5, 0.5, 0.5)
    #     std = (0.5, 0.5, 0.5)
    #     crop_pct = 0.9
    # elif model_type == 'swin':
    #     mean = (0.485, 0.456, 0.406)
    #     std = (0.229, 0.224, 0.225)
    #     crop_pct = 0.9
    # else:
    #     raise NotImplementedError

    # train_transform = build_transform(mean=mean, std=std, crop_pct=crop_pct)
    # val_transform = build_transform(mean=mean, std=std, crop_pct=crop_pct)

    # # Data
    # traindir = os.path.join(args.data, 'train')
    # valdir = os.path.join(args.data, 'val')

    # val_dataset = datasets.ImageFolder(valdir, val_transform)
    # val_loader = torch.utils.data.DataLoader(
    #     val_dataset,
    #     batch_size=args.val_batchsize,
    #     shuffle=False,
    #     num_workers=args.num_workers,
    #     pin_memory=True,
    # )
    
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    trainset = datasets.CIFAR100(root="./data",
                                train=True,
                                download=True,
                                transform=transform_train)
    testset = datasets.CIFAR100(root="./data",
                                train=False,
                                download=True,
                                transform=transform_test)
    
    train_sampler = RandomSampler(trainset)
    test_sampler = SequentialSampler(testset)
    train_loader = DataLoader(trainset,
                              sampler=train_sampler,
                              batch_size=128,
                              num_workers=4,
                              pin_memory=True)
    
    test_loader = DataLoader(testset,
                             sampler=test_sampler,
                             batch_size=args.val_batchsize,
                             num_workers=4,
                             pin_memory=True)

    # switch to evaluate mode
    model.eval()

    # define loss function (criterion)
    criterion = nn.CrossEntropyLoss().to(device)

    if args.quant:
        # train_dataset = datasets.ImageFolder(traindir, train_transform)
        # train_loader = torch.utils.data.DataLoader(
        #     train_dataset,
        #     batch_size=args.calib_batchsize,
        #     shuffle=True,
        #     num_workers=args.num_workers,
        #     pin_memory=True,
        #     drop_last=True,
        # )
        # Get calibration set.
        image_list = []
        for i, (data, target) in enumerate(train_loader):
            if i == args.calib_iter:
                break
            data = data.to(device)
            image_list.append(data)

        print("Calibrating...")
        model.model_open_calibrate()
        with torch.no_grad():
            for i, image in enumerate(image_list):
                if i == len(image_list)-1:
                    # This is used for OMSE method to
                    # calculate minimum quantization error
                    model.model_open_last_calibrate()
                output = model(image)
        model.model_close_calibrate()
        model.model_quant()

    print("Validating...")
    val_loss, val_prec1 = validate(
        args, test_loader, model, criterion, device
    )


def validate(args, val_loader, model, criterion, device):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    val_start_time = end = time.time()
    for i, (data, target) in enumerate(val_loader):
        target = target.to(device)
        data = data.to(device)
        target = target.to(device)

        with torch.no_grad():
            output = model(data)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec1,_ = accuracy(output.data, target, topk=(1,5))
        losses.update(loss.data.item(), data.size(0))
        top1.update(prec1.data.item(), data.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print(
                "Test: [{0}/{1}]\t"
                "Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t"
                "Loss {loss.val:.4f} ({loss.avg:.4f})\t"
                "Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t".format(
                    i,
                    len(val_loader),
                    batch_time=batch_time,
                    loss=losses,
                    top1=top1,
                )
            )
    val_end_time = time.time()
    print(" * Prec@1 {top1.avg:.3f} Time {time:.3f}".format(
        top1=top1, time=val_end_time - val_start_time))

    return losses.avg, top1.avg


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def build_transform(input_size=224, interpolation="bicubic",
                    mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),
                    crop_pct=0.875):
    def _pil_interp(method):
        if method == "bicubic":
            return Image.BICUBIC
        elif method == "lanczos":
            return Image.LANCZOS
        elif method == "hamming":
            return Image.HAMMING
        else:
            return Image.BILINEAR
    resize_im = input_size > 32
    t = []
    if resize_im:
        size = int(math.floor(input_size / crop_pct))
        ip = _pil_interp(interpolation)
        t.append(
            transforms.Resize(
                size, interpolation=ip
            ),  # to maintain same ratio w.r.t. 224 images
        )
        t.append(transforms.CenterCrop(input_size))

    t.append(transforms.ToTensor())
    t.append(transforms.Normalize(mean, std))
    return transforms.Compose(t)


if __name__ == "__main__":
    main()


Model: vit_base_patch16_224


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Calibrating...
Validating...
Test: [0/100]	Time 1.421 (1.421)	Loss 0.3890 (0.3890)	Prec@1 85.000 (85.000)	
 * Prec@1 84.870 Time 88.349
